# Optimization of Radiation Therapy Intensity

One way to treat cancer is **radiation therapy**,which involves using an external beam treatment machine to pass ionizing radiation through the patient’s body, damaging both cancerous and healthy tissues. Normally, several beams are precisely administered from different angles in a two-dimensional plane. Due to attenuation, each beam delivers more radiation to the tissue near the entry point than to the tissue near the exit point. Scatter also causes some delivery of radiation to tissue outside the direct path of the beam.

Since tumor cells are typically microscopically interspersed among healthy cells, the radiation dosage throughout the tumor region must be large enough to kill the malignant cells. At the same time, the aggregate dose to critical tissues must not exceed established tolerance levels, in order to prevent complications that can be more serious than the disease itself. For the same reason, the total dose to the entire healthy anatomy must be minimized.

### Objective

The goal of this design is to select the intensity of each beam to generate the best possible dose distribution. (The dose strength at any point in the body is measured in units called **kilorads**.) Once the treatment design has been developed, it is administered in multiple installments, spread over several weeks.

### Model and Constraints

For any proposed beam of given intensity, analyzing the resulting radiation absorption by various parts of the body requires a complex process. In brief, based on careful anatomical analysis, the energy distribution within the two-dimensional cross section of the tissue can be plotted on an **isodose map**, where the contour lines represent the dose strength as a percentage of the dose strength at the entry point. A fine grid is then placed over the isodose map. By summing the radiation absorbed in the squares containing each type of tissue, the average dose that is absorbed by the tumor, healthy anatomy, and critical tissues can be calculated.

After thorough analysis, the medical team has carefully estimated the data needed, summarized as follows. The first column lists the areas of the body that must be considered, and then the next two columns give the fraction of the radiation dose at the entry point for each beam that is absorbed by the respective areas on average. For example, if the dose level at the entry point for beam 1 is 1 kilorad, then an average of 0.4 kilorad will be absorbed by the entire healthy anatomy in the two-dimensional plane, an average of 0.3 kilorad will be absorbed by nearby critical tissues, an average of 0.5 kilorad will be absorbed by various parts of the tumor, and 0.6 kilorad will be absorbed by the center of the tumor. 

The last column provides dosage restrictions for each area of the body:

- The average dosage absorption for the healthy anatomy must be as small as possible.
- The critical tissues must not exceed 2.7 kilorads.
- The average over the entire tumor must equal 6 kilorads.
- The center of the tumor must be at least 6 kilorads.

### Mathematical Model

Let $x_1$ and $x_2$ represent the intensities of beams 1 and 2, respectively, measured in kilorads. The mathematical optimization model can be structured as follows:

**Objective Function (Minimize radiation absorbed by healthy tissues):**
$$\text{Minimize } 0.4 x_1 + 0.5 x_2$$

**Subject to the following constraints:**

1. **Critical Tissues:**
   $$0.3 x_1 + 0.1 x_2 \leq 2.7$$

2. **Entire Tumor Region:**
   $$0.5 x_1 + 0.5 x_2 = 6$$

3. **Center of Tumor:**
   $$0.6 x_1 + 0.4 x_2 \geq 6$$
   
4. **Non-negativity constraints:**
   $$x_1 \geq 0, \quad x_2 \geq 0 $$

   
This model aims to determine the optimal intensity levels of the two beams such that the absorbed dose by healthy tissues is minimized, while meeting all constraints for critical tissues, the tumor region, and the tumor center.


In [ ]:
!pip install pyomo
!conda install -c conda-forge pyomo coincbc -y

In [8]:
import os
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals

os.environ["PATH"] += ":/Users/polina.esipenko/anaconda3/bin"

model = ConcreteModel()

model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

model.objective = Objective(expr=0.4 * model.x1 + 0.5 * model.x2, sense=1)  # sense=1 == mimimization

""" Adding Constraints """
# Critical Tissues Constraint
model.critical_tissues = Constraint(expr=0.3 * model.x1 + 0.1 * model.x2 <= 2.7)

# Tumour Region Constraint
model.tumor_region = Constraint(expr=0.5 * model.x1 + 0.5 * model.x2 == 6)

# Center of Tumour Constraint
model.center_tumor = Constraint(expr=0.6 * model.x1 + 0.4 * model.x2 >= 6)

# Choosing Solver
solver = SolverFactory('cbc')
result = solver.solve(model)

print("Optimal intensity for Beam 1 (x1):", model.x1.value)
print("Optimal intensity for Beam 2 (x2):", model.x2.value)
print("Minimum dose to healthy tissue:", model.objective.expr())


Optimal intensity for Beam 1 (x1): 7.5
Optimal intensity for Beam 2 (x2): 4.5
Minimum dose to healthy tissue: 5.25
